In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

2024-04-16 18:25:19.302212: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-16 18:25:20.112858: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
titanic_file_path = tf.keras.utils.get_file(
    "train.csv",
    'https://storage.googleapis.com/tf-datasets/titanic/train.csv'
)

df = pd.read_csv(titanic_file_path)

df.head()

,survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,0,male,22.0,1,0,7.2500,Third,unknown,Southampton,n
1,1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
2,1,female,26.0,0,0,7.9250,Third,unknown,Southampton,y
3,1,female,35.0,1,0,53.1000,First,C,Southampton,n
4,0,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y


In [3]:
df.rename(columns={"survived": "target"}, inplace=True)
np.random.seed(5)

In [4]:
train, val, test = np.split(df.sample(frac=1), [int(0.8*len(df)), int(0.9*len(df))])

train

/home/redloaded/anaconda3/envs/transformers/lib/python3.9/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


,target,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
445,0,male,57.0,0,0,12.3500,Second,unknown,Queenstown,y
230,1,female,31.0,0,2,164.8667,First,C,Southampton,n
289,1,male,39.0,0,0,7.9250,Third,unknown,Southampton,y
622,0,male,28.0,0,0,10.5000,Second,unknown,Southampton,y
361,0,female,37.0,0,0,9.5875,Third,unknown,Southampton,y
...,...,...,...,...,...,...,...,...,...,...
572,0,male,35.0,0,0,10.5000,Second,unknown,Southampton,y
54,0,male,26.0,2,0,8.6625,Third,unknown,Southampton,n
19,0,male,28.0,0,0,7.2250,Third,unknown,Cherbourg,y
609,1,female,27.0,1,0,13.8583,Second,unknown,Cherbourg,n


In [5]:
def df_to_dataset(df, shuffle=True, batch_size=32):
    df = df.copy()
    labels = df.pop('target')
    df = {key: value.values[:, tf.newaxis] for key, value in df.items()}
    ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size = len(df))

    ds = ds.batch(batch_size)
    ds = ds.prefetch(batch_size)

    return ds

In [6]:
def get_normalization_layer(name, dataset):
    normalizer = tf.keras.layers.Normalization(axis=None)
    feature_ds = dataset.map(lambda x, y: x[name])
    normalizer.adapt(feature_ds)
    return normalizer


def get_category_encodeing_layer(name, dataset, dtype, max_tokens=None):
  if dtype == "string":
    index = tf.keras.layers.StringLookup(max_tokens=max_tokens)
  else:
    index = tf.keras.layers.IntegerLookup(max_tokens=max_tokens)

  feature_ds = dataset.map(lambda x, y: x[name])

  index.adapt(feature_ds)

  encoder = tf.keras.layers.CategoryEncoding(
      num_tokens=index.vocabulary_size()
  )

  return lambda feature: encoder(index(feature))

In [7]:
batch_size = 10
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, batch_size=batch_size)
test_ds = df_to_dataset(test, batch_size=batch_size)

2024-04-16 18:25:21.058039: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-16 18:25:21.058405: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [8]:
numerical_cols = ["age", "fare"]
numerical_cat_cols = ["n_siblings_spouses", "parch"]
cat_cols = ["sex", "class", "deck", "embark_town", "alone"]

all_inputs = []
encoded_features = []

for header in numerical_cols:
  numerical_col = tf.keras.Input(shape=(1,), name=header)
  normalization_layer = get_normalization_layer(header, train_ds)
  encoded_numerical_col = normalization_layer(numerical_col)

  all_inputs.append(numerical_col)
  encoded_features.append(encoded_numerical_col)

for header in numerical_cat_cols:
  cat_col = tf.keras.Input(shape=(1, ), name=header, dtype='int64')
  encoding_layer = get_category_encodeing_layer(
      header, train_ds, dtype='int64'
  )
  encoded_cat_col = encoding_layer(cat_col)

  all_inputs.append(cat_col)
  encoded_features.append(encoded_cat_col)


for header in cat_cols:
  cat_col = tf.keras.Input(shape=(1, ), name=header, dtype='string')
  encoding_layer = get_category_encodeing_layer(
      header, train_ds, dtype='string'
  )
  encoded_cat_col = encoding_layer(cat_col)

  all_inputs.append(cat_col)
  encoded_features.append(encoded_cat_col)

2024-04-16 18:25:21.213265: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-04-16 18:25:21.291146: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-04-16 18:25:21.346062: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-04-16 18:25:21.387841: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-04-16 18:25:21.438921: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-04-16 18:25:21.479303: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-04-16 18:25:21.527040: W tensorflow/core/framework/local_rendezvous.cc:404] L

In [9]:
x = tf.keras.layers.concatenate(encoded_features)
x = tf.keras.layers.Dense(32, activation="relu")(x)
x = tf.keras.layers.Dense(8, activation="relu")(x)
x = tf.keras.layers.Dense(4, activation="relu")(x)
x = tf.keras.layers.Dense(2, activation="relu")(x)

outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x)

model = tf.keras.Model(all_inputs, outputs)

model.compile(
    optimizer="adam",
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
    metrics=["accuracy"]
)
tf.keras.utils.plot_model(model)

You must install pydot (`pip install pydot`) for `plot_model` to work.


In [10]:
history = model.fit(
    train_ds, validation_data=val_ds, epochs=50,
    callbacks=[tf.keras.callbacks.TensorBoard("log")]
    )

Epoch 1/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4612 - loss: 0.7227 - val_accuracy: 0.6508 - val_loss: 0.6840
Epoch 2/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6867 - loss: 0.6776 - val_accuracy: 0.6984 - val_loss: 0.6694
Epoch 3/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7495 - loss: 0.6589 - val_accuracy: 0.7619 - val_loss: 0.6313
Epoch 4/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7458 - loss: 0.6345 - val_accuracy: 0.7778 - val_loss: 0.5864
Epoch 5/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7691 - loss: 0.6106 - val_accuracy: 0.8095 - val_loss: 0.5679
Epoch 6/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7794 - loss: 0.5868 - val_accuracy: 0.8254 - val_loss: 0.5570
Epoch 7/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7911 - loss: 0.5653 - val_accuracy: 0.8254 - val_loss: 0.5172
Epoch 8/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7865 - loss: 0.5588 - val_accuracy: 0.8571 - val_loss:

In [41]:
print("Original")
display(df.head())
print("Predicted")
inference = df.head().drop('target', axis=1)
inference = inference[numerical_cols+numerical_cat_cols+cat_cols]
inference['target'] = model(list(inference.values.T))
display(inference[df.columns])

Original


,target,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,0,male,22.0,1,0,7.2500,Third,unknown,Southampton,n
1,1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
2,1,female,26.0,0,0,7.9250,Third,unknown,Southampton,y
3,1,female,35.0,1,0,53.1000,First,C,Southampton,n
4,0,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y


Predicted


,target,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,0.188897,male,22.0,1,0,7.2500,Third,unknown,Southampton,n
1,0.999886,female,38.0,1,0,71.2833,First,C,Cherbourg,n
2,0.647986,female,26.0,0,0,7.9250,Third,unknown,Southampton,y
3,0.995191,female,35.0,1,0,53.1000,First,C,Southampton,n
4,0.189391,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y
